# RETRIEVING DATA

In [4]:
!pip install --upgrade numpy

In [5]:
import requests
import pandas as pd

# Space-Track login
USERNAME = "krutiray95@gmail.com"
PASSWORD = "Kxrwebster9439316894!"

session = requests.Session()
login_url = "https://www.space-track.org/ajaxauth/login"
login_payload = {"identity": USERNAME, "password": PASSWORD}
session.post(login_url, data=login_payload)

# Query SATCAT CSV
url = "https://www.space-track.org/basicspacedata/query/class/satcat/orderby/NORAD_CAT_ID asc/format/csv"
response = session.get(url)

# Save CSV locally
with open("satcat.csv", "wb") as f:
    f.write(response.content)

# Load into pandas
df = pd.read_csv("satcat.csv")
print(df.head())
print(len(df), "rows loaded")

     INTLDES  NORAD_CAT_ID  OBJECT_TYPE     SATNAME COUNTRY      LAUNCH  \
0  1957-001A             1  ROCKET BODY    SL-1 R/B     CIS  1957-10-04   
1  1957-001B             2      PAYLOAD   SPUTNIK 1     CIS  1957-10-04   
2  1957-002A             3      PAYLOAD   SPUTNIK 2     CIS  1957-11-03   
3  1958-001A             4      PAYLOAD  EXPLORER 1      US  1958-02-01   
4  1958-002B             5      PAYLOAD  VANGUARD 1      US  1958-03-17   

    SITE       DECAY  PERIOD  INCLINATION  ...  RCSVALUE  RCS_SIZE  FILE  \
0  TTMTR  1957-12-01   96.19        65.10  ...         0     LARGE     1   
1  TTMTR  1958-01-03   96.10        65.00  ...         0       NaN  7179   
2  TTMTR  1958-04-14  103.74        65.33  ...         0     SMALL  9221   
3  AFETR  1970-03-31   88.48        33.15  ...         0       NaN     1   
4  AFETR         NaN  132.61        34.26  ...         0     SMALL  9242   

   LAUNCH_YEAR  LAUNCH_NUM LAUNCH_PIECE  CURRENT  OBJECT_NAME  OBJECT_ID  \
0         1957  

In [6]:
import os, requests, pandas as pd

USER = os.getenv("SPACETRACK_USER", "krutiray95@gmail.com")
PASS = os.getenv("SPACETRACK_PASS", "Kxrwebster9439316894!")

s = requests.Session()
s.post("https://www.space-track.org/ajaxauth/login",
       data={"identity": USER, "password": PASS})

url = ("https://www.space-track.org/basicspacedata/query/"
       "class/decay/DECAY_EPOCH/%3E2020-01-01/"
       "orderby/DECAY_EPOCH asc/format/csv")

r = s.get(url); r.raise_for_status()
open("decay.csv","wb").write(r.content)

df = pd.read_csv("decay.csv")
print(df.head()); print(len(df), "rows loaded")

   NORAD_CAT_ID  OBJECT_NUMBER     OBJECT_NAME     INTLDES   OBJECT_ID  RCS  \
0         44179          44179  MICROSAT-R DEB  2019-006BS  2019-006BS    0   
1         44179          44179  MICROSAT-R DEB  2019-006BS  2019-006BS    0   
2         26476          26476           DM-F3   2000-048A   2000-048A    0   
3         26476          26476           DM-F3   2000-048A   2000-048A    0   
4         44117          44117  MICROSAT-R DEB   2019-006C   2019-006C    0   

  RCS_SIZE COUNTRY            MSG_EPOCH         DECAY_EPOCH     SOURCE  \
0    SMALL     IND  2020-12-24 21:50:16  2020-01-01 0:00:00     satcat   
1    SMALL     IND  2020-01-06 20:44:00  2020-01-01 0:00:00  decay_msg   
2    LARGE      US  2019-12-29 09:02:00  2020-01-01 0:59:00    TIP_msg   
3    LARGE      US  2019-12-30 03:58:00  2020-01-01 4:08:00    TIP_msg   
4    SMALL     IND  2019-11-21 19:40:04  2020-01-02 0:00:00  60day_msg   

     MSG_TYPE  PRECEDENCE  
0  Historical           1  
1  Historical           

# DATA CLEANING PROCESS

## Imports & Paths

In [7]:
import os
import pandas as pd
import numpy as np

# ---- Configure file paths here ----
DATA_DIR = "."  # folder with your CSVs ('.' = current folder)

SATCAT_FILE = os.path.join(DATA_DIR, "satcat.csv")
DECAY_FILE  = os.path.join(DATA_DIR, "decay.csv")

# If you know your TLE filename, set it directly:
# TLE_FILE = os.path.join(DATA_DIR, "my_tle_export.csv")

# Otherwise: auto-detect a Space-Track export that starts with 'st_' or fall back to any csv named 'gp*.csv'
tlcands = [f for f in os.listdir(DATA_DIR) if f.lower().endswith(".csv") and (f.startswith("st_") or f.lower().startswith("gp"))]
TLE_FILE = os.path.join(DATA_DIR, tlcands[0]) if tlcands else None

print("SATCAT_FILE:", SATCAT_FILE)
print("DECAY_FILE :", DECAY_FILE)
print("TLE_FILE   :", TLE_FILE)

SATCAT_FILE: .\satcat.csv
DECAY_FILE : .\decay.csv
TLE_FILE   : .\st_krutiray95@gmail_com_20250831_1445368442.csv


## Utility Helpers

In [17]:
def clean_cols(df):
    df = df.copy()
    df.columns = [c.strip().upper() for c in df.columns]
    return df

def parse_dt(series):
    return pd.to_datetime(series, utc=True, errors="coerce")

def num(series):
    return pd.to_numeric(series, errors="coerce")

def save(df, path):
    df.to_csv(path, index=False)
    print(f"✔ Saved: {path}  ({len(df):,} rows)")

## Cleaning SATCAT (metadata)

In [9]:
# Load
satcat = pd.read_csv(SATCAT_FILE, low_memory=False)
satcat = clean_cols(satcat)

# Keep typical useful columns if present
keep_satcat = [
    "NORAD_CAT_ID","SATNAME","INTLDES","TYPE","COUNTRY",
    "LAUNCH","DECAY","PERIOD","INCL","APOGEE","PERIGEE","RCS"
]
satcat = satcat[[c for c in keep_satcat if c in satcat.columns]].copy()

# Types
if "NORAD_CAT_ID" in satcat: satcat["NORAD_CAT_ID"] = num(satcat["NORAD_CAT_ID"]).astype("Int64")
if "LAUNCH" in satcat: satcat["LAUNCH"] = parse_dt(satcat["LAUNCH"])
if "DECAY"  in satcat: satcat["DECAY"]  = parse_dt(satcat["DECAY"])
for c in ["PERIOD","INCL","APOGEE","PERIGEE","RCS"]:
    if c in satcat.columns: satcat[c] = num(satcat[c])

# Quality: drop null IDs and dedupe to one row per NORAD
satcat = satcat.dropna(subset=["NORAD_CAT_ID"]).drop_duplicates("NORAD_CAT_ID")

# Save
SATCAT_CLEAN = os.path.join(DATA_DIR, "satcat_clean.csv")
save(satcat, SATCAT_CLEAN)

# Quick QA
display(satcat.head())
print("\nNull rates:\n", satcat.isna().mean().sort_values(ascending=False).head(8))

✔ Saved: .\satcat_clean.csv  (65,265 rows)


,NORAD_CAT_ID,SATNAME,INTLDES,COUNTRY,LAUNCH,DECAY,PERIOD,APOGEE,PERIGEE
0,1,SL-1 R/B,1957-001A,CIS,1957-10-04 00:00:00+00:00,1957-12-01 00:00:00+00:00,96.19,938.0,214.0
1,2,SPUTNIK 1,1957-001B,CIS,1957-10-04 00:00:00+00:00,1958-01-03 00:00:00+00:00,96.10,1080.0,64.0
2,3,SPUTNIK 2,1957-002A,CIS,1957-11-03 00:00:00+00:00,1958-04-14 00:00:00+00:00,103.74,1659.0,211.0
3,4,EXPLORER 1,1958-001A,US,1958-02-01 00:00:00+00:00,1970-03-31 00:00:00+00:00,88.48,215.0,183.0
4,5,VANGUARD 1,1958-002B,US,1958-03-17 00:00:00+00:00,NaT,132.61,3823.0,648.0



Null rates:
 DECAY           0.4792
PERIOD          0.0152
PERIGEE         0.0152
APOGEE          0.0152
NORAD_CAT_ID    0.0000
LAUNCH          0.0000
COUNTRY         0.0000
INTLDES         0.0000
dtype: float64


## Cleaning DECAY/REENTRY (Space-Track class/decay)

In [10]:
decay = pd.read_csv(DECAY_FILE, low_memory=False)
decay = clean_cols(decay)

# Common columns in 'decay' class:
keep_decay = [
    "NORAD_CAT_ID","OBJECT_NAME","INTLDES","COUNTRY","RCS",
    "MSG_EPOCH","DECAY_EPOCH","SOURCE","TYPE","ELSET"
]
decay = decay[[c for c in keep_decay if c in decay.columns]].copy()

# Types
if "NORAD_CAT_ID" in decay: decay["NORAD_CAT_ID"] = num(decay["NORAD_CAT_ID"]).astype("Int64")
for c in ["MSG_EPOCH","DECAY_EPOCH"]:
    if c in decay.columns: decay[c] = parse_dt(decay[c])
if "RCS" in decay.columns: decay["RCS"] = num(decay["RCS"])

# Clean
decay = decay.dropna(subset=["NORAD_CAT_ID"]).drop_duplicates()

# Save
DECAY_CLEAN = os.path.join(DATA_DIR, "decay_clean.csv")
save(decay, DECAY_CLEAN)

# QA
display(decay.head())
print("\nDate range (DECAY_EPOCH):", decay.get("DECAY_EPOCH", pd.Series(dtype="datetime64[ns, UTC]")).min(),
      "→", decay.get("DECAY_EPOCH", pd.Series(dtype="datetime64[ns, UTC]")).max())

✔ Saved: .\decay_clean.csv  (63,694 rows)


,NORAD_CAT_ID,OBJECT_NAME,INTLDES,COUNTRY,RCS,MSG_EPOCH,DECAY_EPOCH,SOURCE
0,44179,MICROSAT-R DEB,2019-006BS,IND,0,2020-12-24 21:50:16+00:00,2020-01-01 00:00:00+00:00,satcat
1,44179,MICROSAT-R DEB,2019-006BS,IND,0,2020-01-06 20:44:00+00:00,2020-01-01 00:00:00+00:00,decay_msg
2,26476,DM-F3,2000-048A,US,0,2019-12-29 09:02:00+00:00,2020-01-01 00:59:00+00:00,TIP_msg
3,26476,DM-F3,2000-048A,US,0,2019-12-30 03:58:00+00:00,2020-01-01 04:08:00+00:00,TIP_msg
4,44117,MICROSAT-R DEB,2019-006C,IND,0,2019-11-21 19:40:04+00:00,2020-01-02 00:00:00+00:00,60day_msg



Date range (DECAY_EPOCH): 2020-01-01 00:00:00+00:00 → 2025-10-25 00:00:00+00:00


## Cleaning TLE/GP and derive features

In [11]:
if TLE_FILE is None:
    raise FileNotFoundError("No TLE CSV found. Set TLE_FILE above or place a file (e.g. 'st_*.csv') next to the notebook.")

gp = pd.read_csv(TLE_FILE, low_memory=False)
gp = clean_cols(gp)

# Normalize common alternative column names if needed
rename_map = {
    "MEANMOTION":"MEAN_MOTION",
    "ECC":"ECCENTRICITY",
    "BSTARDRAG":"BSTAR",
    "INCL":"INCLINATION",
}
for k,v in rename_map.items():
    if k in gp.columns and v not in gp.columns:
        gp = gp.rename(columns={k:v})

# Expected core columns—filter to what's available
expected_gp = [
    "NORAD_CAT_ID","EPOCH","MEAN_MOTION","ECCENTRICITY","BSTAR","INCLINATION",
    "MEAN_ANOMALY","ARG_OF_PERICENTER","RA_OF_ASC_NODE","TLE_LINE1","TLE_LINE2"
]
gp = gp[[c for c in expected_gp if c in gp.columns]].copy()

# Types
if "NORAD_CAT_ID" in gp: gp["NORAD_CAT_ID"] = num(gp["NORAD_CAT_ID"]).astype("Int64")
if "EPOCH" in gp: gp["EPOCH"] = parse_dt(gp["EPOCH"])
for c in ["MEAN_MOTION","ECCENTRICITY","BSTAR","INCLINATION","MEAN_ANOMALY","ARG_OF_PERICENTER","RA_OF_ASC_NODE"]:
    if c in gp.columns: gp[c] = num(gp[c])

# Drop bad rows, sort, de-dupe key (NORAD,EPOCH)
gp = gp.dropna(subset=["NORAD_CAT_ID","EPOCH"]).sort_values(["NORAD_CAT_ID","EPOCH"])
gp = gp.drop_duplicates(subset=["NORAD_CAT_ID","EPOCH"], keep="last").reset_index(drop=True)

# Sanity bounds (adjust to your regime)
if "ECCENTRICITY" in gp: gp = gp[(gp["ECCENTRICITY"] >= 0) & (gp["ECCENTRICITY"] < 1)]
if "MEAN_MOTION"  in gp: gp = gp[(gp["MEAN_MOTION"] > 0) & (gp["MEAN_MOTION"] < 25)]

# Derive semi-major axis & altitudes (km)
MU = 398600.4418  # km^3/s^2
RE_KM = 6378.137
if {"MEAN_MOTION","ECCENTRICITY"}.issubset(gp.columns):
    n_rad_s = gp["MEAN_MOTION"] * 2*np.pi / 86400.0
    a_km = (MU / (n_rad_s**2)) ** (1/3)
    e = gp["ECCENTRICITY"].clip(lower=0, upper=0.999999)
    rp_km = a_km * (1 - e)
    ra_km = a_km * (1 + e)
    gp["A_KM"]  = a_km
    gp["HP_KM"] = rp_km - RE_KM  # perigee altitude
    gp["HA_KM"] = ra_km - RE_KM  # apogee altitude

# Save cleaned GP
GP_CLEAN = os.path.join(DATA_DIR, "gp_clean.csv")
save(gp, GP_CLEAN)

# QA
display(gp.head())
print("\nCounts per satellite (top 5):")
display(gp["NORAD_CAT_ID"].value_counts().head())

✔ Saved: .\gp_clean.csv  (9,571 rows)


,NORAD_CAT_ID,EPOCH,MEAN_MOTION,ECCENTRICITY,BSTAR,INCLINATION,MEAN_ANOMALY,ARG_OF_PERICENTER,RA_OF_ASC_NODE,TLE_LINE1,TLE_LINE2,A_KM,HP_KM,HA_KM
0,25544,2020-01-01 06:11:10.983264+00:00,15.495303,0.000505,0.000034,51.6446,3.7257,90.7711,99.4171,1 25544U 98067A 20001.25776601 .00001469 0...,2 25544 51.6446 99.4171 0005055 90.7711 3...,6796.236040,414.663542,421.534537
1,25544,2020-01-01 12:23:48.583392+00:00,15.495288,0.000514,0.000024,51.6442,7.2280,91.7041,98.1370,1 25544U 98067A 20001.51653453 .00000882 0...,2 25544 51.6442 98.1370 0005144 91.7041 7...,6796.240557,414.607571,421.599543
2,25544,2020-01-01 14:05:26.578752+00:00,15.495300,0.000534,0.000030,51.6440,40.6973,92.2059,97.7889,1 25544U 98067A 20001.58711318 .00001214 0...,2 25544 51.6440 97.7889 0005337 92.2059 40...,6796.236990,414.472838,421.727142
3,25544,2020-01-01 17:17:49.889184+00:00,15.495307,0.000534,0.000031,51.6443,65.8338,92.8411,97.1279,1 25544U 98067A 20001.72071631 .00001295 0...,2 25544 51.6443 97.1279 0005339 92.8411 65...,6796.235092,414.469582,421.726602
4,25544,2020-01-01 18:47:02.666400+00:00,15.495305,0.000518,0.000028,51.6444,51.8986,92.6004,96.8194,1 25544U 98067A 20001.78266975 .00001093 0...,2 25544 51.6444 96.8194 0005179 92.6004 51...,6796.235648,414.578877,421.618418



Counts per satellite (top 5):


NORAD_CAT_ID
25544    9571
Name: count, dtype: Int64

## daily time series + simple anomaly flags

In [12]:
def build_daily(gp):
    out = []
    for norad, g in gp.groupby("NORAD_CAT_ID"):
        g = g.set_index("EPOCH").sort_index()
        g = g[~g.index.duplicated(keep="last")]           # remove duplicate timestamps
        cols = [c for c in ["BSTAR","MEAN_MOTION","HP_KM","HA_KM","A_KM"] if c in g.columns]
        if not cols: 
            continue
        daily = g[cols].resample("1D").interpolate("time") # daily interpolation
        daily["NORAD_CAT_ID"] = norad
        out.append(daily.reset_index())
    return pd.concat(out, ignore_index=True) if out else pd.DataFrame(columns=["EPOCH","NORAD_CAT_ID"])

gp_daily = build_daily(gp)

# Rolling 7-day median/STD z-scores (tune thresholds later)
if "MEAN_MOTION" in gp_daily:
    mm_med = gp_daily.groupby("NORAD_CAT_ID")["MEAN_MOTION"].transform(lambda s: s.rolling(7, min_periods=3).median())
    mm_std = gp_daily.groupby("NORAD_CAT_ID")["MEAN_MOTION"].transform(lambda s: s.rolling(7, min_periods=3).std(ddof=0))
    gp_daily["MM_Z"] = (gp_daily["MEAN_MOTION"] - mm_med) / (mm_std + 1e-9)

if "BSTAR" in gp_daily:
    b_med = gp_daily.groupby("NORAD_CAT_ID")["BSTAR"].transform(lambda s: s.rolling(7, min_periods=3).median())
    b_std = gp_daily.groupby("NORAD_CAT_ID")["BSTAR"].transform(lambda s: s.rolling(7, min_periods=3).std(ddof=0))
    gp_daily["BSTAR_Z"] = (gp_daily["BSTAR"] - b_med) / (b_std + 1e-9)

if {"MM_Z","BSTAR_Z"}.issubset(gp_daily.columns):
    gp_daily["ANOMALY_FLAG"] = (gp_daily["MM_Z"] > 2.0) | (gp_daily["BSTAR_Z"] > 2.0)

GP_DAILY = os.path.join(DATA_DIR, "gp_daily_features.csv")
save(gp_daily, GP_DAILY)

display(gp_daily.head())
print("\nDaily rows:", len(gp_daily))

✔ Saved: .\gp_daily_features.csv  (1,827 rows)


,EPOCH,BSTAR,MEAN_MOTION,HP_KM,HA_KM,A_KM,NORAD_CAT_ID,MM_Z,BSTAR_Z,ANOMALY_FLAG
0,2020-01-01 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,25544,NaN,NaN,False
1,2020-01-02 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,25544,NaN,NaN,False
2,2020-01-03 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,25544,NaN,NaN,False
3,2020-01-04 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,25544,NaN,NaN,False
4,2020-01-05 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,25544,NaN,NaN,False



Daily rows: 1827


## Joining daily orbital features with SATCAT metadata

In [14]:
meta_cols = [c for c in ["NORAD_CAT_ID","SATNAME","TYPE","COUNTRY","LAUNCH","DECAY"] if c in satcat.columns]
joined = gp_daily.merge(satcat[meta_cols], on="NORAD_CAT_ID", how="left")

JOINED_FILE = os.path.join(DATA_DIR, "orbital_daily_with_meta.csv")
save(joined, JOINED_FILE)

display(joined.head())

✔ Saved: .\orbital_daily_with_meta.csv  (1,827 rows)


,EPOCH,BSTAR,MEAN_MOTION,HP_KM,HA_KM,A_KM,NORAD_CAT_ID,MM_Z,BSTAR_Z,ANOMALY_FLAG,SATNAME,COUNTRY,LAUNCH,DECAY
0,2020-01-01 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,25544,NaN,NaN,False,ISS (ZARYA),ISS,1998-11-20 00:00:00+00:00,NaT
1,2020-01-02 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,25544,NaN,NaN,False,ISS (ZARYA),ISS,1998-11-20 00:00:00+00:00,NaT
2,2020-01-03 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,25544,NaN,NaN,False,ISS (ZARYA),ISS,1998-11-20 00:00:00+00:00,NaT
3,2020-01-04 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,25544,NaN,NaN,False,ISS (ZARYA),ISS,1998-11-20 00:00:00+00:00,NaT
4,2020-01-05 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,25544,NaN,NaN,False,ISS (ZARYA),ISS,1998-11-20 00:00:00+00:00,NaT


In [15]:
# Duplicates in key
dups = gp.duplicated(subset=["NORAD_CAT_ID","EPOCH"]).sum()
print("Duplicate (NORAD,EPOCH) rows in gp_clean:", dups)

# Ranges
if "ECCENTRICITY" in gp: print("Ecc min/max:", gp["ECCENTRICITY"].min(), gp["ECCENTRICITY"].max())
if "MEAN_MOTION" in gp:  print("MM min/max:", gp["MEAN_MOTION"].min(), gp["MEAN_MOTION"].max())

# Missingness snapshot
print("\nMissingness (gp_clean):\n", gp.isna().mean().sort_values(ascending=False).head(8))

Duplicate (NORAD,EPOCH) rows in gp_clean: 0
Ecc min/max: 3e-06 0.0013583
MM min/max: 15.48249857 15.51817712

Missingness (gp_clean):
 NORAD_CAT_ID         0.0
EPOCH                0.0
MEAN_MOTION          0.0
ECCENTRICITY         0.0
BSTAR                0.0
INCLINATION          0.0
MEAN_ANOMALY         0.0
ARG_OF_PERICENTER    0.0
dtype: float64
